In [3]:
import pandas as pd

In [35]:
def process_transcript(transcript_path):
    transcript_df = pd.read_csv(transcript_path)

    course_categories = {
        'General Education': {
            'English Composition & Comprehension': 3,
            'English Composition & Comprehension Lab': 1,
            'Introduction to Info. & Comm. Technologies': 3,
            'Communication & Presentation Skills': 3,
            'Professional Practices': 3,
            'Technical & Business Writing': 3,
            'Pakistan Studies': 3,
            'Islamic Studies & Ethics': 3
        },

        'University Elective': {
            'Foreign Language': 2,
            'Social Service': 1,
            'Management Related': 3,
            'Social Science Related': 3,
            'Economy Related': 3,
            'Entrepreneurship': 3,
            'Arabic Language': 3,
            'Psychology': 3,
            'Micro Economics': 3,
            'International HR Mgmt': 3,
            'Fund of Management': 3
        },

        'Mathematics & Science Foundation': {
            'Calculus & Analytical Geometry': 3,
            'Applied Physics': 3,
            'Probability & Statistics': 3,
            'Linear Algebra': 3,
            'Basic Electronics': 3,
            'Calculus - I': 3,
            'Calculus - II': 3
        },

        'Computing – Core': {
            'Programming Fundamentals': 3,
            'Programming Fundamentals Lab': 1,
            'Object Oriented Programming': 3,
            'Object Oriented Programming Lab': 1,
            'Data Structures & Algorithms': 3,
            'Data Structures & Algorithms Lab': 1,
            'Discrete Structures': 3,
            'Database Systems': 3,
            'Database Systems Lab': 3,
            'Operating Systems': 3,
            'Operating Systems Lab': 1,
            'Software Engineering': 3,
            'Computer Networks': 3,
            'Computer Networks Lab': 1,
            'Information Security': 3,
            'Final Year Project': 3,
            'Human Comp Int': 3
        },

        'Domain CS Core': {
            'Digital Logic Design': 3,
            'Digital Logic Design Lab': 1,
            'Comp. Organization & Assembly Language': 3,
            'Comp. Organization & Assembly Language Lab': 1,
            'Design & Analysis of Algorithms': 3,
            'Theory of Automata': 3,
            'Compiler Construction': 3,
            'Compiler Construction Lab': 1,
            'Artificial Intelligence': 3,
            'Artificial Intelligence Lab': 1,
            'Parallel & Distributed Computing': 3,
            'Computer Architecture': 3,
            'OOAD Lab': 1,
            'OOAD': 3
        },

        'Domain CS Elective': {
            'CS Elective': 3,
            'CS Elective 1': 3,
            'CS Elective 2': 3,
            'CS Elective 3': 3,
            'CS Elective 4': 3,
            'CS Elective 5': 3,
            'Dsgn Dfct and Restrct': 3,
            'Intro. to SW PM': 3,
            'Software Testing': 3,
            'IS Audit and Control': 3
        },

        'Domain CS Supporting': {
            'Differential Equations': 3,
            'Multi-variate Calculus': 3,
            'Graph Theory': 3,
            'Theory of Programming Languages': 3,
            'Fund.of Probabilis Programming': 3,
            'Numerical Computing': 3
        }
    }

    requirements = {
        'General Education': 19,
        'University Elective': 12,
        'Mathematics & Science Foundation': 12,
        'Computing – Core': 39,
        'Domain CS Core': 24,
        'Domain CS Elective': 15,
        'Domain CS Supporting': 9
    }

    alternative_names_mapping = {
        'Introduction to Info. & Comm. Technologies': ['Introduction to ICT', 'Intro to Info. & Comm. Technologies'],
        'Islamic Studies & Ethics': ['Islamic Studies/ Ethics', 'Islam and Relig Std'],
        'English Composition & Comprehension': ['English Language'],
        'English Composition & Comprehension Lab': ['English Language Lab'],
        'Object Oriented Programming': ['Computer Programming'],
        'Object Oriented Programming Lab': ['Computer Prog Lab'],
        'Programming Fundamentals': ['Intro to Computing'],
        'Programming Fundamentals Lab': ['Intro to Comp Lab'],
        'Digital Logic Design Lab': ['DLD Lab'],
        'Final Year Project': ['Project - I', 'Project - II'],
        'Design & Analysis of Algorithms': ['DA of Algorithms'],
        'Computer Architecture': ['Comp Architecture'],
        'Comp. Organization & Assembly Language': ['COAL'],
        'Comp. Organization & Assembly Language Lab': ['COAL Lab'],
        'Technical & Business Writing': ['TBW'],
        'Probability & Statistics': ['Probability and Stat'],
        'Operating Systems Lab': ['Operating Sys Lab'],
        'Computer Networks Lab': ['Comp Networks Lab'],
        'Information Security': ['Info Sys Security'],
        'Professional Practices': ['Prof Issues in IT'],
        'Communication & Presentation Skills': ['English Composition']
    }

    def is_course_completed(grade, points):
        return grade in ['A+', 'A', 'A-', 'B+', 'B', 'B-', 'C+', 'C', 'C-', 'D+', 'D', 'D-']

    completed_categories_credit_hours = {category: 0 for category in course_categories}
    completed_categories_courses = {category: [] for category in course_categories}
    final_year_project_count = 0
    courses_taken = {}
    generic = {'Domain CS Core',
               'Domain CS Core Lab',
               'CS Elective',
               'Domain CS Supporting',
               'Foreign Language',
               'Social Service',
               'Management Related',
               'Social Science Related',
               'Economy Related'}

    for index, row in transcript_df.iterrows():
        course_name = row['courseName']
        credit_hours = row['creditHour']
        grade = row['grade']
        points = row['points']

        for original_name, alternative_names in alternative_names_mapping.items():
            if course_name in alternative_names:
                course_name = original_name

        for category, courses in course_categories.items():
            if course_name in courses:
                hec_credit_hours = courses[course_name]
                if course_name == 'Final Year Project':
                    if final_year_project_count < 2 and is_course_completed(grade, points):
                        completed_categories_credit_hours[category] += credit_hours
                        completed_categories_courses[category].append({
                            'courseName': course_name,
                            'UniversitycreditHour': credit_hours,
                            'HECCreditHour': hec_credit_hours,
                            'grade': grade
                        })
                        final_year_project_count += 1
                elif course_name in generic:
                    if is_course_completed(grade, points):
                        completed_categories_credit_hours[category] += credit_hours
                        completed_categories_courses[category].append({
                            'courseName': course_name,
                            'creditHour': credit_hours,
                            'HECCreditHour': hec_credit_hours,
                            'grade': grade
                        })
                elif is_course_completed(grade, points) and course_name not in courses_taken:
                    completed_categories_credit_hours[category] += credit_hours
                    if course_name in courses_taken:
                        completed_categories_courses[category] = [
                            entry for entry in completed_categories_courses[category]
                            if entry != courses_taken[course_name]
                        ]
                    completed_categories_courses[category].append({
                        'courseName': course_name,
                        'creditHour': credit_hours,
                        'HECCreditHour': hec_credit_hours,
                        'grade': grade
                    })
                    courses_taken[course_name] = {
                        'courseName': course_name,
                        'creditHour': credit_hours,
                        'HECCreditHour': hec_credit_hours,
                        'grade': grade
                    }

    print('\t\t\t\t\t\t', 'Analysis of the Transcript:')
    for category, required_courses in course_categories.items():
        print('\n', '-' * 120, '\n')
        print('\t\t\t\t',
              f'{category}: {completed_categories_credit_hours[category]}/{requirements[category]} credit hours completed',
              '\n')

        if completed_categories_courses[category]:
            print(f'Completed {category} Courses:')
            completed_courses_df = pd.DataFrame(completed_categories_courses[category])
            print(completed_courses_df.to_string(index=False))
        else:
            print('No completed courses in this category.')

    print('\n', '_' * 120, '\n')
    is_degree_valid = all(
        completed_categories_credit_hours[category] >= requirements[category] for category in course_categories)
    print('\t\t\t\t', f'The student {"has" if is_degree_valid else "has not"} completed all degree requirements.')
    total_credit_hours = sum(completed_categories_credit_hours.values())
    print('\t\t\t\t\t', f'Total Credit Hours Completed: {total_credit_hours}/{130}')
    print('\n', '_' * 120, '\n')

    for category, required_courses in course_categories.items():
        if completed_categories_credit_hours[category] < requirements[category]:
            remaining_credit_hours = requirements[category] - completed_categories_credit_hours[category]
            print(f'{category}: {remaining_credit_hours} credit hours remaining')
            print('Remaining Courses:')
            remaining_courses = set(required_courses) - set(
                course['courseName'] for course in completed_categories_courses[category])
            print(', '.join(remaining_courses))
            print('\n')

In [36]:
process_transcript(r'C:\Users\UBL-HO.DESKTOP-7ET3E40\Desktop\BAI-5A\KRR\P1 - HEC Degree Requiremnts\Raw Data\Test Std-0102.csv')

						 Analysis of the Transcript:

 ------------------------------------------------------------------------------------------------------------------------ 

				 General Education: 19/19 credit hours completed 

Completed General Education Courses:
                             courseName  creditHour  HECCreditHour grade
    English Composition & Comprehension           3              3     A
English Composition & Comprehension Lab           1              1     A
               Islamic Studies & Ethics           3              3    A-
                       Pakistan Studies           3              3    B+
    Communication & Presentation Skills           3              3     A
           Technical & Business Writing           3              3     B
                 Professional Practices           3              3    B+

 ------------------------------------------------------------------------------------------------------------------------ 

				 University Elective: 15/12 credit 